In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8-darkgrid")

In [2]:
required_cols = {"open", "high", "low", "close", "signal"}
assert required_cols.issubset(set(ohlc.columns)), "OHLC data missing required columns"

ohlc = ohlc.reset_index(drop=True)

NameError: name 'ohlc' is not defined

In [ ]:
position = 0  
entry_price = 0

trades = []
equity = [1.0]

In [ ]:
for i in range(1, len(ohlc)):
    signal = ohlc.loc[i - 1, "signal"]
    open_price = ohlc.loc[i, "open"]

    # Enter long
    if signal == "BUY" and position == 0:
        position = 1
        entry_price = open_price

    # Exit long
    elif signal == "SELL" and position == 1:
        pnl = (open_price - entry_price) / entry_price
        trades.append(pnl)
        equity.append(equity[-1] * (1 + pnl))
        position = 0

    else:
        equity.append(equity[-1])

In [ ]:
equity = pd.Series(equity)

plt.figure(figsize=(12, 4))
plt.plot(equity)
plt.title("Equity Curve (Vision-Based Backtest)")
plt.xlabel("Trades / Time")
plt.ylabel("Equity")
plt.show()

In [ ]:
rolling_max = equity.cummax()
drawdown = (equity - rolling_max) / rolling_max

plt.figure(figsize=(12, 4))
plt.plot(drawdown, color="red")
plt.title("Drawdown")
plt.show()

In [ ]:
trades = np.array(trades)

total_trades = len(trades)
win_rate = (trades > 0).mean() if total_trades > 0 else 0
avg_win = trades[trades > 0].mean() if np.any(trades > 0) else 0
avg_loss = trades[trades < 0].mean() if np.any(trades < 0) else 0
expectancy = trades.mean() if total_trades > 0 else 0
max_drawdown = drawdown.min()

In [ ]:
if trades.std() > 0:
    sharpe = trades.mean() / trades.std()
else:
    sharpe = 0

In [ ]:
print("--- Backtest Report ---")
print("=" * 55)

print(f"Total Trades     : {total_trades}")
print(f"Win Rate         : {win_rate:.2%}")
print(f"Expectancy       : {expectancy:.4f}")
print(f"Average Win      : {avg_win:.4f}")
print(f"Average Loss     : {avg_loss:.4f}")
print(f"Sharpe (proxy)   : {sharpe:.2f}")
print(f"Max Drawdown     : {max_drawdown:.2%}")
print(f"Final Equity     : {equity.iloc[-1]:.2f}")